In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import geopandas as gpd
import geopy
import keplergl
import overpy
import joblib
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

#%matplotlib inline

In [2]:
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon

## Read data

In [3]:
train = pd.read_csv('mf_geo_train.csv')
test = pd.read_csv('mf_geo_test.csv')

In [4]:
train.head(249)

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206
2,ZgodVRqB,39.635721,47.213330,-1.039679
3,0t2jNYdz,37.704570,55.782020,-1.169339
4,U27W4QJ7,37.643983,55.730188,-0.088837
...,...,...,...,...
244,h6qIFhXA,93.010041,56.020784,0.905225
245,zrqrRf0q,55.986665,54.718073,-0.650698
246,p4OinUCs,39.690907,47.218739,-0.953239
247,ZEvDCFD6,37.563670,55.744340,0.948446


In [5]:
test.head(2)

,point_id,lon,lat,target
0,F4lXR1cG,37.681242,55.74804,0.0091
1,4LJu4GTf,60.580910,56.79586,0.0091


## City


In [6]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="astrowa")

lat, lon = 39.635721,	47.213330
#print('lat: ', lat)
#print('lon: ', lon)
coord = str(lat) + str(', ')	+ str(lon)
location = geolocator.reverse('55.750271,	37.784210')
print(location.address)#37.784210	55.750271 39.635721,	47.213330

38/18, 2-я Владимирская улица, Новогиреево, район Новогиреево, Москва, Центральный федеральный округ, 4А, Россия


In [7]:
location.raw

{'place_id': 218413796,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 620857829,
 'lat': '55.75032525',
 'lon': '37.784413883027824',
 'display_name': '38/18, 2-я Владимирская улица, Новогиреево, район Новогиреево, Москва, Центральный федеральный округ, 4А, Россия',
 'address': {'house_number': '38/18',
  'road': '2-я Владимирская улица',
  'suburb': 'Новогиреево',
  'city_district': 'район Новогиреево',
  'city': 'Москва',
  'state': 'Москва',
  'region': 'Центральный федеральный округ',
  'postcode': '4А',
  'country': 'Россия',
  'country_code': 'ru'},
 'boundingbox': ['55.7501234', '55.7505401', '37.7840144', '37.784735']}

In [8]:
locationer_city = []
lats, lons= train['lat'][3],train['lon'][3]
coords = str(lats) + str(', ') + str(lons)
locationer = geolocator.reverse(coords)
if locationer.raw['address']['state'] == 'Москва':
    locationer_city.append(locationer.raw['address']['state'])
else:
    try:
        locationer_city.append(locationer.raw['address']['city'])
    except:
        locationer_city.append(locationer.raw['address']['state'])

In [9]:
locationer.raw

{'place_id': 104305978,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 49922415,
 'lat': '55.78199535',
 'lon': '37.704070004582576',
 'display_name': 'Торговый центр "Серебряный Дом", 16, Большая Семёновская улица, Семёновское, район Соколиная Гора, Москва, Центральный федеральный округ, 105094, Россия',
 'address': {'shop': 'Торговый центр "Серебряный Дом"',
  'house_number': '16',
  'road': 'Большая Семёновская улица',
  'suburb': 'Семёновское',
  'city_district': 'район Соколиная Гора',
  'city': 'Москва',
  'state': 'Москва',
  'region': 'Центральный федеральный округ',
  'postcode': '105094',
  'country': 'Россия',
  'country_code': 'ru'},
 'boundingbox': ['55.7816782', '55.7822771', '37.703284', '37.7048441']}

In [10]:
location_city = []#, scheme = 'https'
geolocator = Nominatim(user_agent="astrowa", timeout=100)
for i in range(len(train['lon'])):
    lats, lons= train['lat'][i],	train['lon'][i]
    coords = str(lats) + str(', ') + str(lons)
    location = geolocator.reverse(coords)
    if location.raw['address']['state'] == 'Москва':
        location_city.append(location.raw['address']['state'])
    else:
        try:
            location_city.append(location.raw['address']['city'])
        except:
            location_city.append(location.raw['address']['state'])
    print(location.address)

6А, Сигнальный проезд, район Отрадное, Москва, Центральный федеральный округ, 127106, Россия
38/18, 2-я Владимирская улица, Новогиреево, район Новогиреево, Москва, Центральный федеральный округ, 4А, Россия
18, улица Зорге, ЗЖМ, Советский район, Ростов-на-Дону, городской округ Ростов-на-Дону, Ростовская область, Южный федеральный округ, 344000, Россия
Торговый центр "Серебряный Дом", 16, Большая Семёновская улица, Семёновское, район Соколиная Гора, Москва, Центральный федеральный округ, 105094, Россия
5, Кожевническая улица, район Замоскворечье, Москва, Центральный федеральный округ, 115280, Россия
12, улица 9 Мая, Северный, 3-ий микрорайон, Советский район, Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660000, Россия
Ледовый дворец, 1, проспект Пятилеток, округ Правобережный, Санкт-Петербург, Северо-Западный федеральный округ, 193318, Россия
Торгово-развлекательный центр "Калейдоскоп", 56, Сходненская улица, район Южное Тушино, Москва, Централь

133, Красноармейская улица, Шмидта, Железнодорожный район, городской округ Самара, Самарская область, Приволжский федеральный округ, 443028, Россия
27, улица Академика Королёва, Сипайлово, Октябрьский район, Уфа, городской округ Уфа, Башкортостан, Приволжский федеральный округ, 450000, Россия
Мегафон, проспект Большевиков, Невский округ, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Мегафон, улица Татарстан, Старо-Татарская слобода, Вахитовский район, городской округ Казань, Татарстан, Приволжский федеральный округ, 420021, Россия
167, улица Фрунзе, Ягодная слобода, Кировский район, городской округ Казань, Татарстан, Приволжский федеральный округ, 420033, Россия
Кировский район, Самара, Самарская область, Приволжский федеральный округ, 443028, Россия
129, улица Карла Маркса, Центральный район, Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660049, Россия
Тандем, 56, проспект Ибрагимова, Кизическая слобода, Московский район, 

ТРК «Международный», 3, улица Белы Куна, округ Купчино, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
31, Русаковская улица, Сокольники, район Сокольники, Москва, Центральный федеральный округ, 101113, Россия
ТК Пик, 2, улица Ефимова, Сенной округ, Санкт-Петербург, Северо-Западный федеральный округ, 190031, Россия
Торгово-развлекательный центр "Океания", 57, Кутузовский проспект, Давыдково, район Фили-Давыдково, Москва, Центральный федеральный округ, 121352, Россия
РИО, 1, Большая Черёмушкинская улица, район Котловка, Москва, Центральный федеральный округ, 117447, Россия
Торговый центр, Академическая площадь, микрорайон В, городской округ Троицк, Москва, Центральный федеральный округ, 142191, Россия
Мегафон, проспект Ленина, Рабочий городок, Октябрьский район, Ростов-на-Дону, городской округ Ростов-на-Дону, Ростовская область, Южный федеральный округ, 344018, Россия
19, Удельный проспект, Удельная, округ Светлановское, Санкт-Петербург, Северо-Западный федеральный о

Торговый центр «Ока», 8, Октябрьская улица, Простоквашино, Колпино, Санкт-Петербург, Северо-Западный федеральный округ, 196655, Россия
45, улица 60 лет Октября, Свердловский район, Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660000, Россия
72А, улица Юлиуса Фучика, Горки-2, Приволжский район, городской округ Казань, Татарстан, Приволжский федеральный округ, 420141, Россия
23, Пролетарский проспект, район Царицыно, Москва, Центральный федеральный округ, 115447, Россия
19А, Тверская улица, 49, Тверской район, Москва, Центральный федеральный округ, 125009, Россия
21, улица Грибоедова, Химмаш, Чкаловский район, Екатеринбург, городской округ Екатеринбург, Свердловская область, Уральский федеральный округ, 620010, Россия
1, улица Бетанкура, Канавинский район, Нижний Новгород, городской округ Нижний Новгород, Нижегородская область, Приволжский федеральный округ, 603159, Россия
101/22, проспект Ленина, Дачный, Ворошиловский район, Ростов-на-Дону, гор

Главный вокзал, 1/2, Привокзальная площадь, Железнодорожный район, Ростов-на-Дону, городской округ Ростов-на-Дону, Ростовская область, Южный федеральный округ, 344082, Россия
2, Брянская улица, район Дорогомилово, Москва, Центральный федеральный округ, 121059, Россия
Decathlon, 5, Советская площадь, Советский район, Нижний Новгород, городской округ Нижний Новгород, Нижегородская область, Приволжский федеральный округ, 603122, Россия
43-45, Лиговский проспект, Владимирский округ, Санкт-Петербург, Северо-Западный федеральный округ, 191040, Россия
Лента, 11, улица Коминтерна, Московский район, Нижний Новгород, городской округ Нижний Новгород, Нижегородская область, Приволжский федеральный округ, 603157, Россия
ТЦ Дирижабль, 17, улица Академика Шварца, Созвездие, Ботанический, Чкаловский район, Екатеринбург, городской округ Екатеринбург, Свердловская область, Уральский федеральный округ, 620073, Россия
ТРЦ Планета, 20, улица Энтузиастов, Октябрьский район, Уфа, городской округ Уфа, Башкорт

Станция метро «Нарвская», проспект Стачек, Нарвский округ, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
ТРК «Европолис», 84, Полюстровский проспект, Выборгская сторона, округ Сампсониевское, Санкт-Петербург, Северо-Западный федеральный округ, 195197, Россия
Кировский район, Уфа, городской округ Уфа, Башкортостан, Приволжский федеральный округ, 450000, Россия
50 к1, Шипиловская улица, Зябликово, район Зябликово, Москва, Центральный федеральный округ, 109387, Россия
МегаФон, Люблинская улица, Марьинский парк, район Марьино, Москва, Центральный федеральный округ, 109369, Россия
Гудзон, 12, Каширское шоссе, район Нагатино-Садовники, Москва, Центральный федеральный округ, 115230, Россия
Торговый центр "Тук-Тук", 2, Матвеевская улица, Матвеевское, район Очаково-Матвеевское, Москва, Центральный федеральный округ, 119517, Россия
56/72, Щёлковское шоссе, микрорайон 1 Мая, Балашиха, городской округ Балашиха, Московская область, Центральный федеральный округ, 143904, Россия


15Б, улица Декабристов, район Отрадное, Москва, Центральный федеральный округ, 127566, Россия
36 к2, проспект Ветеранов, Дачное, округ Дачное, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
53, проспект Металлургов, Советский район, Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, 660133, Россия
ТЦ "Аврора", 47А, Аэродромная улица, Железнодорожный район, городской округ Самара, Самарская область, Приволжский федеральный округ, 443070, Россия
18, проспект Дзержинского, Дзержинский район, Новосибирск, Новосибирская область, Сибирский федеральный округ, 630000, Россия
168, Московский проспект, округ Московская застава, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Тимирязевская, Дмитровское шоссе, Соломенная сторожка, Тимирязевский район, Москва, Центральный федеральный округ, 127343, Россия
248/1, улица Бориса Богаткова, Дзержинский район, городской округ Новосибирск, Новосибирская область, Сибирский федераль

In [11]:
len(location_city)

425

In [12]:
location_city

['Москва',
 'Москва',
 'Ростов-на-Дону',
 'Москва',
 'Москва',
 'Красноярск',
 'Санкт-Петербург',
 'Москва',
 'Красноярск',
 'Ростов-на-Дону',
 'Ростов-на-Дону',
 'Москва',
 'Уфа',
 'городской округ Казань',
 'Москва',
 'Москва',
 'Екатеринбург',
 'Нижний Новгород',
 'Красноярск',
 'Москва',
 'Колпино',
 'Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Красноярск',
 'Ростов-на-Дону',
 'городской округ Новосибирск',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Москва',
 'Нижний Новгород',
 'Москва',
 'Санкт-Петербург',
 'Санкт-Петербург',
 'городской округ Самара',
 'Ростов-на-Дону',
 'Нижний Новгород',
 'Екатеринбург',
 'Москва',
 'Санкт-Петербург',
 'городской округ Новосибирск',
 'Москва',
 'Москва',
 'Санкт-Петербург',
 'Москва',
 'Москва',
 'Москва',
 'Санкт-Петербург',
 'Уфа',
 'городской округ Казань',
 'Москва',
 'Екатеринбург',
 'городской округ Самара',
 'Уфа',
 'Санкт-Петербург',
 'городск

In [13]:
train['city'] = location_city

In [14]:
train.head(15)

,point_id,lon,lat,target,city
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва
2,ZgodVRqB,39.635721,47.213330,-1.039679,Ростов-на-Дону
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва
4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва
5,ci9r9Fr2,92.926002,56.065908,-0.218497,Красноярск
6,O3tOF6uE,30.468500,59.921400,-1.039679,Санкт-Петербург
7,CO76tdVs,37.444304,55.850511,0.213704,Москва
8,GU0QclY1,92.903162,56.050290,-0.304937,Красноярск
9,CHtHXJLp,39.701709,47.234012,-0.132057,Ростов-на-Дону


In [15]:
train['city'].unique()

array(['Москва', 'Ростов-на-Дону', 'Красноярск', 'Санкт-Петербург', 'Уфа',
       'городской округ Казань', 'Екатеринбург', 'Нижний Новгород',
       'Колпино', 'Новосибирск', 'городской округ Новосибирск',
       'городской округ Самара', 'Самара', 'Казань', 'Балашиха', 'Пушкин'],
      dtype=object)

In [16]:
train['city'].value_counts()

Москва                         160
Санкт-Петербург                 78
городской округ Самара          22
Екатеринбург                    22
Нижний Новгород                 21
Ростов-на-Дону                  21
Красноярск                      20
Уфа                             19
городской округ Казань          17
городской округ Новосибирск     14
Новосибирск                     12
Казань                           8
Самара                           5
Колпино                          4
Пушкин                           1
Балашиха                         1
Name: city, dtype: int64

In [17]:
train['city'][train['city'] == "Пушкин"] = 'Санкт-Петербург'

In [18]:
train['city'][train['city'] == "Балашиха"] = 'Москва'

In [19]:
train['city'][train['city'] == "городской округ Самара"] = 'Самара'

In [20]:
train['city'][train['city'] == "городской округ Казань"] = 'Казань'

In [21]:
train['city'][train['city'] == "городской округ Новосибирск"] = 'Новосибирск'

In [22]:
train['city'][train['city'] == "Колпино"] = 'Санкт-Петербург'

In [23]:
train['city'].value_counts()

Москва             161
Санкт-Петербург     83
Самара              27
Новосибирск         26
Казань              25
Екатеринбург        22
Нижний Новгород     21
Ростов-на-Дону      21
Красноярск          20
Уфа                 19
Name: city, dtype: int64

In [24]:
towner = train['city'].unique()

In [25]:
towner = tuple(towner)

In [26]:
towner

('Москва',
 'Ростов-на-Дону',
 'Красноярск',
 'Санкт-Петербург',
 'Уфа',
 'Казань',
 'Екатеринбург',
 'Нижний Новгород',
 'Новосибирск',
 'Самара')

In [27]:
import overpass
import requests

# Получаем данные по станциям

In [28]:
overpass_query = f"""
[out:json];
area[name="{'Москва'}"];
(node["railway"="station"](area);
 way["railway"="station"](area);
 rel["railway"="station"](area);
);
out center;
"""
response = requests.get("http://overpass-api.de/api/interpreter", 
                        params={'data': overpass_query})
data = response.json()

In [31]:
for i in range(len(towner)):#почему-то работает не с первого раза
    overpass_query = f"""
    [out:json];
    area[name="{towner[i]}"];
    (node["railway"="station"](area);
     way["railway"="station"](area);
     rel["railway"="station"](area);
    );
    out center;
    """
    response = requests.get("http://overpass-api.de/api/interpreter", 
                            params={'data': overpass_query}, timeout=1000000)
    data[towner[i]] = response.json()
    print(towner[i])

Москва
Ростов-на-Дону
Красноярск
Санкт-Петербург
Уфа
Казань
Екатеринбург
Нижний Новгород
Новосибирск
Самара


In [32]:
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.56.7 b85c4387',
 'osm3s': {'timestamp_osm_base': '2020-12-21T12:16:03Z',
  'timestamp_areas_base': '2020-12-21T12:00:03Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 26999673,
   'lat': 55.7788343,
   'lon': 37.6537207,
   'tags': {'alt_name': 'Москва-Октябрьская',
    'esr:user': '060073',
    'loc_name': 'Москва-Ленинградская',
    'loc_name:website': 'http://www.tutu.ru/station.php?nnst=79310',
    'name': 'Москва-Пассажирская',
    'name:esr': 'Москва-Пассажирская',
    'nat_name': 'Москва',
    'official_name': 'Москва-Пассажирская',
    'official_name:esr': 'Москва-Пассажирская',
    'official_name:esr:website': 'http://cargo.rzd.ru/cargostation/public/ru?STRUCTURE_ID=5101&layer_id=4829&page4821_2705=1&refererLayerId=4821&id=1090',
    'official_name:express-3': 'Москва-Октябрьская',
    'official_name:express-3:

In [33]:
data['Уфа']

{'version': 0.6,
 'generator': 'Overpass API 0.7.56.7 b85c4387',
 'osm3s': {'timestamp_osm_base': '2020-12-21T12:22:03Z',
  'timestamp_areas_base': '2020-12-21T12:00:03Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 727305011,
   'lat': 54.7806071,
   'lon': 56.222392,
   'tags': {'esr:user': '655304',
    'name': 'Шакша',
    'name:ru': 'Шакша',
    'public_transport': 'station',
    'railway': 'station',
    'uic_ref': '2024742'}},
  {'type': 'node',
   'id': 1307944304,
   'lat': 54.8066228,
   'lon': 56.0976068,
   'tags': {'name': 'Черниковка', 'railway': 'station', 'uic_ref': '2024738'}},
  {'type': 'node',
   'id': 1327126088,
   'lat': 54.8370987,
   'lon': 56.1203711,
   'tags': {'name': 'Черниковка-Восточная',
    'railway': 'station',
    'uic_ref': '2024208'}},
  {'type': 'node',
   'id': 1327126090,
   'lat': 54.8126836,
   'lon': 56.1370539,
   'tags': {

In [34]:
data.keys()

dict_keys(['version', 'generator', 'osm3s', 'elements', 'Москва', 'Ростов-на-Дону', 'Красноярск', 'Санкт-Петербург', 'Уфа', 'Казань', 'Екатеринбург', 'Нижний Новгород', 'Новосибирск', 'Самара'])

In [35]:
data['Москва']['elements'][0]['tags']

{'alt_name': 'Москва-Октябрьская',
 'esr:user': '060073',
 'loc_name': 'Москва-Ленинградская',
 'loc_name:website': 'http://www.tutu.ru/station.php?nnst=79310',
 'name': 'Москва-Пассажирская',
 'name:esr': 'Москва-Пассажирская',
 'nat_name': 'Москва',
 'official_name': 'Москва-Пассажирская',
 'official_name:esr': 'Москва-Пассажирская',
 'official_name:esr:website': 'http://cargo.rzd.ru/cargostation/public/ru?STRUCTURE_ID=5101&layer_id=4829&page4821_2705=1&refererLayerId=4821&id=1090',
 'official_name:express-3': 'Москва-Октябрьская',
 'official_name:express-3:website': 'http://bilet.ru/rus/TrainDirectory.htm?firstsymb=%u041c',
 'official_name:website': 'http://pass.rzd.ru/timetable/public/ru?STRUCTURE_ID=5104&layer_id=5368&id=278&node_id=19',
 'public_transport': 'station',
 'railway': 'station',
 'uic_name': 'Moskva Oktiabrskaia',
 'uic_ref': '2006004'}

In [36]:
datal = pd.DataFrame.from_dict(data['Москва']['elements'])

In [37]:
datal

,type,id,lat,lon,tags
0,node,26999673,55.778834,37.653721,"{'alt_name': 'Москва-Октябрьская', 'esr:user':..."
1,node,31447943,55.726868,37.449885,"{'alt_name': 'Кунцево', 'esr:user': '181704', ..."
2,node,60660466,55.887177,37.661550,"{'colour': 'orange', 'layer': '-1', 'name': 'М..."
3,node,60660469,55.869625,37.664184,"{'colour': 'orange', 'layer': '-2', 'name': 'Б..."
4,node,61088523,55.757774,37.662184,"{'express:user': '2000001', 'loc_name': 'Курск..."
...,...,...,...,...,...
298,node,7339988431,55.705025,37.819000,"{'colour': '#FFCBDB', 'layer': '-3', 'name': '..."
299,node,7340764979,55.764922,37.706704,"{'colour': '#FFCBDB', 'layer': '-3', 'name': '..."
300,node,7340764980,55.753643,37.719203,"{'colour': '#FFCBDB', 'layer': '-3', 'name': '..."
301,node,7340764981,55.731780,37.730031,"{'colour': '#FFCBDB', 'layer': '-3', 'name': '..."


In [38]:
train.head(2)

,point_id,lon,lat,target,city
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва


In [39]:
train[(train['lat'] > 55.77) & (train['lon'] > 37.65) & (train['city'] == 'Москва')]

,point_id,lon,lat,target,city
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва
14,Nj7qV6KG,37.799786,55.793427,0.473024,Москва
33,XTKhDNze,37.659739,55.845203,-1.104509,Москва
35,kXV2cKsB,37.660087,55.843941,-0.477818,Москва
38,eqECPIIT,37.738100,55.815425,-0.304937,Москва
54,c5wVfiUF,37.679968,55.772930,1.812847,Москва
74,VdNzagUv,37.678141,55.789394,1.250986,Москва
76,EYeTJlnp,37.653334,55.856351,0.040823,Москва
79,UT4p9zvj,37.659371,55.776859,-0.434598,Москва
91,s8LhdRDp,37.723067,55.782563,-0.261717,Москва


In [40]:
delta = train[train['city'] == 'Москва']

In [41]:
delta.index

Int64Index([  0,   1,   3,   4,   7,  11,  14,  15,  19,  21,
            ...
            406, 408, 410, 414, 417, 420, 421, 422, 423, 424],
           dtype='int64', length=161)

In [42]:
len(delta)

161

In [43]:
from geopy import distance

In [44]:
towner

('Москва',
 'Ростов-на-Дону',
 'Красноярск',
 'Санкт-Петербург',
 'Уфа',
 'Казань',
 'Екатеринбург',
 'Нижний Новгород',
 'Новосибирск',
 'Самара')

In [45]:
pd.DataFrame.from_dict(data['Красноярск']['elements'])

,type,id,lat,lon,tags
0,node,248606294,56.042417,92.763066,"{'name': 'Бугач', 'public_transport': 'station..."
1,node,248606397,55.998447,92.951680,"{'name': 'Злобино', 'name:en': 'Zlobino', 'pub..."
2,node,248606407,55.980004,92.841601,"{'esr:user': '891401', 'name': 'Енисей', 'name..."
3,node,458941735,55.972812,92.775682,"{'esr:user': '891435', 'name': 'Красноярские С..."
4,node,1858215626,56.008706,92.855392,"{'name': 'Юбилейная (детская ж/д)', 'railway':..."
5,node,2792164837,56.073157,92.933153,"{'esr:user': '890108', 'int_name': 'Krasnoyars..."
6,node,3294562636,55.982343,92.867419,"{'name': 'Белые росы', 'railway': 'station'}"
7,node,3299954202,56.042411,92.762692,"{'name': 'Бугач', 'public_transport': 'station..."
8,node,4017600946,56.022095,93.065467,"{'esr:user': '892103', 'name': 'Базаиха', 'nam..."
9,node,4535206674,56.005901,92.829035,"{'esr:user': '890004', 'name': 'Красноярск-Пас..."


In [46]:
for town in (towner):
    datal = pd.DataFrame.from_dict(data[town]['elements'])
    delta = train[train['city'] == town]

In [47]:
#train.head(45)

In [48]:
delta#поинты магазинов

,point_id,lon,lat,target,city
44,FS7Ufln2,50.215690,53.239159,0.386584,Самара
62,Mk1D78ip,50.126074,53.187828,-1.169339,Самара
67,3uG2b8Ye,50.263857,53.218848,2.634029,Самара
81,P2LhIYVH,50.123711,53.192857,0.991666,Самара
103,Ccqm3lC6,50.262682,53.220259,3.109450,Самара
118,myPybSTm,50.181802,53.212513,-0.348157,Самара
134,bBxa5jY2,50.148747,53.217111,-0.477818,Самара
139,sMvuaVtL,50.221690,53.248670,0.948446,Самара
158,tkDyf0LP,50.173675,53.140380,1.596747,Самара
169,LwZsJ4Cy,50.190391,53.191808,0.905225,Самара


In [49]:
datal#поинты рэилвеев

,type,id,lat,lon,tags
0,node,351119704,53.212474,50.282584,"{'name': 'Юнгородок', 'name:cs': 'Jungorodok',..."
1,node,351119750,53.212475,50.247656,"{'name': 'Безымянка', 'name:cs': 'Bezymjanka',..."
2,node,442008171,53.206983,50.235770,"{'name': 'Победа', 'name:cs': 'Poběda', 'name:..."
3,node,442008172,53.201554,50.220441,"{'name': 'Советская', 'name:cs': 'Sovětskaja',..."
4,node,442008173,53.200956,50.199578,"{'name': 'Спортивная', 'name:cs': 'Sportivnaja..."
5,node,442008174,53.200274,50.176512,"{'name': 'Гагаринская', 'name:cs': 'Gagarinska..."
6,node,442008175,53.202809,50.160496,"{'name': 'Московская', 'name:cs': 'Moskovskaja..."
7,node,442008176,53.211918,50.148986,"{'name': 'Российская', 'name:cs': 'Rossijskaja..."
8,node,457012018,53.185685,50.121301,"{'esr:user': '657907', 'name': 'Самара', 'name..."
9,node,596477731,53.509278,50.140436,"{'name': 'Аэропорт ""Курумоч""', 'name:en': 'Air..."


In [50]:
def delta_range(datal, delta):#считаем минимальные расстояния от магазинов до станций

    outputer = []
    indexer = []
    for elem in (delta.index):
        a_coords = (delta['lat'][elem], delta['lon'][elem])
        #indexer.append(prom.index(min(prom)))
        prom = []
        for i in range(len(datal)):
            b_coords = (datal['lat'][i], datal['lon'][i])
            prom.append(distance.great_circle(a_coords, b_coords).m)
            
        outputer.append(min(prom))
        indexer.append(elem)

    return outputer, indexer

In [51]:
outputer = []
indexs = []
for town in (towner):
    datal = pd.DataFrame.from_dict(data[town]['elements'])
    delta = train[train['city'] == town]
    print(town)
    outputer_, indexs_ = delta_range(datal, delta)
    outputer.append(outputer_)
    indexs.append(indexs_)

Москва
Ростов-на-Дону
Красноярск
Санкт-Петербург
Уфа
Казань
Екатеринбург
Нижний Новгород
Новосибирск
Самара


In [52]:
delta

,point_id,lon,lat,target,city
44,FS7Ufln2,50.215690,53.239159,0.386584,Самара
62,Mk1D78ip,50.126074,53.187828,-1.169339,Самара
67,3uG2b8Ye,50.263857,53.218848,2.634029,Самара
81,P2LhIYVH,50.123711,53.192857,0.991666,Самара
103,Ccqm3lC6,50.262682,53.220259,3.109450,Самара
118,myPybSTm,50.181802,53.212513,-0.348157,Самара
134,bBxa5jY2,50.148747,53.217111,-0.477818,Самара
139,sMvuaVtL,50.221690,53.248670,0.948446,Самара
158,tkDyf0LP,50.173675,53.140380,1.596747,Самара
169,LwZsJ4Cy,50.190391,53.191808,0.905225,Самара


In [53]:
def coun_shop_in_range(rails, shop):#количество магазинов в радиусе 1000 метров
    
    distance_dicts = dict()
    distance_min = dict()
    for elem in (shop.index):
        a_coords = (shop['lat'][elem], shop['lon'][elem])
        distance_dicts[elem] = 0
        for i in range(len(rails)):
            b_coords = (rails['lat'][i], rails['lon'][i])
            ranges = distance.great_circle(a_coords, b_coords).m
            if ranges < 1000:
                distance_dicts[elem] += 1 #количество рэилвеев до 500
                
    return distance_dicts

In [54]:
dicter_count = dict()
for town in (towner):
    datal = pd.DataFrame.from_dict(data[town]['elements'])
    delta = train[train['city'] == town]
    print(town)
    _count = coun_shop_in_range(datal, delta)
    dicter_count[town] = _count

Москва
Ростов-на-Дону
Красноярск
Санкт-Петербург
Уфа
Казань
Екатеринбург
Нижний Новгород
Новосибирск
Самара


In [55]:
dicter_count.keys()

dict_keys(['Москва', 'Ростов-на-Дону', 'Красноярск', 'Санкт-Петербург', 'Уфа', 'Казань', 'Екатеринбург', 'Нижний Новгород', 'Новосибирск', 'Самара'])

In [56]:
distance_shop = pd.DataFrame()

In [57]:
distance_shop['Distance'] = outputer

In [58]:
distance_shop['index'] = indexs

In [59]:
train[train['city'] == 'Москва'].head(10)

,point_id,lon,lat,target,city
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва
4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва
7,CO76tdVs,37.444304,55.850511,0.213704,Москва
11,XPqFCntc,37.437191,55.664705,-0.564258,Москва
14,Nj7qV6KG,37.799786,55.793427,0.473024,Москва
15,E2fZAexI,37.404780,55.810240,-0.564258,Москва
19,e5ZywFU6,37.730449,55.706608,0.386584,Москва
21,wXDJYllD,37.597414,55.850995,0.645905,Москва


In [60]:
len(distance_shop['index'][0])

161

In [61]:
features_1 = []
features_2 = []
features_3 = []
features_4 = []
features_5 = []
x=0
for j in range(10):
    for i in range(len(distance_shop['Distance'][j])):
        if distance_shop['Distance'][j][i] < 100:
            print(distance_shop['Distance'][j][i], distance_shop['index'][j][i])
            features_1.append(distance_shop['index'][j][i])
            x +=1
        if distance_shop['Distance'][j][i] < 500:
            print(distance_shop['Distance'][j][i], distance_shop['index'][j][i])
            features_2.append(distance_shop['index'][j][i])
        if distance_shop['Distance'][j][i] < 1000:
            print(distance_shop['Distance'][j][i], distance_shop['index'][j][i])
            features_3.append(distance_shop['index'][j][i])
print(x)

159.14216878174346 0
159.14216878174346 0
175.58465030215612 1
175.58465030215612 1
59.2492770463366 3
59.2492770463366 3
59.2492770463366 3
221.2142898177973 4
221.2142898177973 4
282.79465821324 7
282.79465821324 7
969.3321518914504 11
146.54118453218686 14
146.54118453218686 14
735.3009888963965 15
257.5273784113034 19
257.5273784113034 19
532.5449859398472 21
50.33222595991769 28
50.33222595991769 28
50.33222595991769 28
808.0009929141798 29
96.4997689469963 30
96.4997689469963 30
96.4997689469963 30
459.40161037235634 31
459.40161037235634 31
182.48809568422695 32
182.48809568422695 32
758.9044361370717 33
162.8145823564817 34
162.8145823564817 34
635.8547982278278 35
46.526107678564244 36
46.526107678564244 36
46.526107678564244 36
163.979925778191 37
163.979925778191 37
184.92082960777364 38
184.92082960777364 38
289.98876816542884 39
289.98876816542884 39
127.43302039200924 41
127.43302039200924 41
542.33138224522 48
163.25388188287053 51
163.25388188287053 51
176.9369079698159

In [62]:
#features_1

In [63]:
len(distance_shop['Distance'][3])

83

In [64]:
nulls = 0

In [65]:
#train['X5'] = nulls

In [66]:
train = train.join(pd.DataFrame(
    {
        'X1': nulls,
        'X2': nulls,
        'X3': nulls,
        'V_Rad_600': nulls,
        'distance_min': nulls,
        'dist_centr': float(nulls),
    }, index=train.index
))

In [67]:
train

,point_id,lon,lat,target,city,X1,X2,X3,V_Rad_600,distance_min,dist_centr
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва,0,0,0,0,0,0.0
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва,0,0,0,0,0,0.0
2,ZgodVRqB,39.635721,47.213330,-1.039679,Ростов-на-Дону,0,0,0,0,0,0.0
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва,0,0,0,0,0,0.0
4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
420,AA8hN7bJ,37.628765,55.740664,0.213704,Москва,0,0,0,0,0,0.0
421,uNw6t6xk,37.586825,55.794233,-0.002397,Москва,0,0,0,0,0,0.0
422,Ap42ei8k,37.678790,55.772910,-0.910019,Москва,0,0,0,0,0,0.0
423,rn9A8r62,37.752130,55.619640,-0.326547,Москва,0,0,0,0,0,0.0


In [68]:
#train.drop(('X3', 'X4'), axis=1)

In [69]:
fechi = [features_1, features_2, features_3]#добавляем фичи
x = 0
for fis in fechi:
    x+=1
    k = ''
    k = ('X' + str(x))
    for elm in fis:
        train[k][elm] = 1

In [70]:
train['distance_min']

0      0
1      0
2      0
3      0
4      0
      ..
420    0
421    0
422    0
423    0
424    0
Name: distance_min, Length: 425, dtype: int64

In [71]:
for town in (towner):
     for elementser in dicter_count[town]:
            train['V_Rad_600'][elementser] = dicter_count[town][elementser]

In [72]:
for i in range(len(distance_shop['index'])):
    print('i = ',i)
    k = 0
    for els in (distance_shop['index'][i]):#по элементам
        train['distance_min'][els] = distance_shop['Distance'][i][k]
        k+=1

i =  0
i =  1
i =  2
i =  3
i =  4
i =  5
i =  6
i =  7
i =  8
i =  9


In [73]:
distance_shop

,Distance,index
0,"[159.14216878174346, 175.58465030215612, 59.24...","[0, 1, 3, 4, 7, 11, 14, 15, 19, 21, 27, 28, 29..."
1,"[2551.9271876181724, 1940.9385512785916, 2056....","[2, 9, 10, 25, 45, 93, 112, 130, 135, 144, 190..."
2,"[920.213040023936, 3151.509040777747, 2256.649...","[5, 8, 18, 24, 68, 82, 107, 154, 159, 165, 170..."
3,"[255.1191576789176, 2507.259890673998, 162.002...","[6, 20, 22, 42, 43, 49, 53, 57, 64, 70, 71, 75..."
4,"[4845.797662771757, 856.7142198936284, 3909.25...","[12, 58, 63, 94, 115, 155, 179, 201, 203, 218,..."
5,"[1865.7285086572242, 313.95974718108465, 969.7...","[13, 59, 65, 66, 69, 88, 90, 92, 114, 121, 137..."
6,"[1067.2218483627796, 3251.5618554339644, 299.2...","[16, 47, 61, 80, 83, 85, 123, 173, 174, 188, 1..."
7,"[1606.9172555330408, 1641.0988346932415, 122.8...","[17, 40, 46, 105, 132, 156, 162, 172, 177, 189..."
8,"[1324.7150297131093, 165.74991448973518, 723.7...","[23, 26, 50, 100, 122, 136, 138, 141, 147, 151..."
9,"[3651.2782664825545, 397.4297472580653, 882.38...","[44, 62, 67, 81, 103, 118, 134, 139, 158, 169,..."


In [74]:
train.head(2)

,point_id,lon,lat,target,city,X1,X2,X3,V_Rad_600,distance_min,dist_centr
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва,0,1,1,2,159,0.0
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва,0,1,1,1,175,0.0


In [75]:
#trains = train.drop('city', axis=1)

In [76]:
trains = train

# Добавляем ещё фичей

In [77]:
data_centr_points = []

In [78]:
#data_centr_points['УФА'] =  geolocator.geocode(query=city)#получаем центральные поинты

In [79]:
def range_to_centr(shop, centr_points):#расстояние до центра
    dicter_range_to_centr = dict()
    centr_points_coords = centr_points
    for elem in (shop.index):
        shop_coords = (shop['lat'][elem], shop['lon'][elem])
        ranges = distance.great_circle(centr_points_coords, shop_coords).km
        dicter_range_to_centr[elem] = ranges
        
    return dicter_range_to_centr

In [80]:
data_centr_points = dict()
for city in towner:
    data_centr_points[city] =  geolocator.geocode(query= city + ', Россия')#получаем центральные поинты

In [81]:
dict_range_to_centr = dict()
for city in towner:
    shops = train[train['city'] == city]
    dict_range_to_centr[city] = range_to_centr(shops, data_centr_points[city][1])

In [82]:
for town in (towner):
     for elementser in dict_range_to_centr[town]:
            train['dist_centr'][elementser] = dict_range_to_centr[town][elementser]

In [83]:
train[train['city'] == 'Уфа']

,point_id,lon,lat,target,city,X1,X2,X3,V_Rad_600,distance_min,dist_centr
12,1ZvdVY5t,55.993548,54.713414,-0.348157,Уфа,0,0,0,0,4845,3.278583
58,nh41SLK0,56.111306,54.806818,0.862005,Уфа,0,0,1,2,856,13.816586
63,jvXZ5BfY,56.080170,54.772889,-0.477818,Уфа,0,0,0,0,3909,9.975725
94,kcPx2crY,56.075499,54.817080,-1.212559,Уфа,0,0,0,0,1832,13.025278
115,eAjJXePD,55.881095,54.790184,0.862005,Уфа,0,0,0,0,6349,8.298545
155,nFBsJ5k3,55.958351,54.736872,-0.348157,Уфа,0,0,0,0,1428,1.381774
179,bwIuFswf,55.947783,54.725857,-0.521038,Уфа,0,0,0,0,2508,0.036457
201,TPU9VeHE,56.058369,54.771233,-1.363830,Уфа,0,0,0,0,4670,8.704548
203,iL9SqqPf,56.123864,54.779754,-0.780358,Уфа,0,0,0,0,3408,12.791633
218,cj6KcHW8,55.994056,54.697990,-0.002397,Уфа,0,0,0,0,6321,4.329233


In [84]:
mall = dict()#попытка добавить фичу находится ли магазин в ТЦ, но получилось криво
#for i in range(len(towner)):
overpass_query = f"""
[out:json];
area[name="{'Москва'}"];
(node["shop"="mall"](area);
 way["shop"="mall"](area);
 rel["shop"="mall"](area);
);
out body;
"""
tc = requests.get("http://overpass-api.de/api/interpreter", 
                        params={'data': overpass_query}, timeout=1000000)
mall = tc.json()
    #mall[towner[i]] = tc.json()
    #print(towner[i])

In [85]:
len(mall['elements'])

33

In [86]:
#for town in (towner):
    #malls = pd.DataFrame.from_dict(mall[town]['elements'])
    #map_1.add_data(data=malls, name='malls' + str(town))

In [87]:
#malls

In [88]:
map_1 = keplergl.KeplerGl(height=800, width=800)
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=800)

In [89]:
map_1.add_data(data=train, name='train')

In [90]:
for town in (towner):
    datal = pd.DataFrame.from_dict(data[town]['elements'])
    map_1.add_data(data=datal, name=("Railways")+str(town))
    delta = train[train['city'] == town]

In [91]:
import json
json_data = map_1.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)

In [92]:
from sklearn.preprocessing import LabelEncoder

In [93]:
labelencoder = LabelEncoder()

In [94]:
trains.iloc[:, 4] = labelencoder.fit_transform(trains.iloc[:, 4])

In [95]:
trains

,point_id,lon,lat,target,city,X1,X2,X3,V_Rad_600,distance_min,dist_centr
0,ommNZCUV,37.590776,55.848630,-0.348157,3,0,1,1,2,159,11.044544
1,nMe2LHPb,37.784210,55.750271,1.294206,3,0,1,1,1,175,10.433178
2,ZgodVRqB,39.635721,47.213330,-1.039679,6,0,0,0,0,2551,5.786946
3,0t2jNYdz,37.704570,55.782020,-1.169339,3,1,1,1,1,59,6.480453
4,U27W4QJ7,37.643983,55.730188,-0.088837,3,0,1,1,3,221,2.797056
...,...,...,...,...,...,...,...,...,...,...,...
420,AA8hN7bJ,37.628765,55.740664,0.213704,3,1,1,1,4,87,1.296436
421,uNw6t6xk,37.586825,55.794233,-0.002397,3,0,1,1,3,167,5.233129
422,Ap42ei8k,37.678790,55.772910,-0.910019,3,0,1,1,1,113,4.576584
423,rn9A8r62,37.752130,55.619640,-0.326547,3,0,0,1,3,550,16.816207


In [96]:
map_1.add_data(data=mall, name='mall')
#map_1.add_data(data=datal, name=("Railways")+str(town))

# Fit model

In [101]:
X_train, X_valid, y_train, y_valid = train_test_split(trains.drop('target', axis=1), trains[['target']])
model = LinearRegression().fit(X_train.drop('point_id', axis=1), y_train)

In [102]:
mean_absolute_error(y_valid, model.predict(X_valid.drop('point_id', axis=1)))#возможно без городов лучше, но это не точно

0.645170077003828

In [99]:
X_valid

,point_id,lon,lat,city,X1,X2,X3,V_Rad_600,distance_min,dist_centr
265,LjxEMUOO,37.541447,55.655966,3,0,1,1,1,148,11.535762
347,mU5wwKjd,39.765401,47.254866,6,1,1,1,2,61,5.519810
305,2TAJRlVf,30.323314,60.036664,8,0,1,1,1,183,10.896684
348,te8dMz6Z,37.516440,55.778380,3,0,1,1,3,213,7.043616
68,NTjDKBtL,92.865583,56.009936,2,0,0,1,1,648,0.440892
...,...,...,...,...,...,...,...,...,...,...
204,XRbToMaK,50.270990,53.275607,7,0,0,0,0,3512,13.507211
376,zYEPdy6c,82.978310,55.037668,5,1,1,1,1,98,7.951976
386,Y5ARpcKu,30.162239,59.807008,8,0,0,0,0,2086,16.982310
143,mj6Q5LGP,30.145125,59.835831,8,0,0,0,0,2434,14.900835


# Make submission

In [100]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = model.predict(X_valid.drop('point_id', axis=1))
submission.to_csv('my_submission_03.csv', index=False)